In [ ]:
!nvidia-smi

Wed Mar  6 13:05:14 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!lscpu

Architecture:            x86_64
  CPU op-mode(s):        32-bit, 64-bit
  Address sizes:         46 bits physical, 48 bits virtual
  Byte Order:            Little Endian
CPU(s):                  2
  On-line CPU(s) list:   0,1
Vendor ID:               GenuineIntel
  Model name:            Intel(R) Xeon(R) CPU @ 2.30GHz
    CPU family:          6
    Model:               63
    Thread(s) per core:  2
    Core(s) per socket:  1
    Socket(s):           1
    Stepping:            0
    BogoMIPS:            4599.99
    Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clf
                         lush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_
                         good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fm
                         a cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hyp
                         ervisor lahf_lm abm invpcid_single ssbd ibrs ibpb stibp

In [ ]:
!CT_CUBLAS=1 pip install ctransformers --no-binary ctransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.1/376.1 kB 2.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for ctransformers: filename=ctransformers-0.2.27-cp310-cp310-linux_x86_64.whl size=2336830 sha256=702f527e62f5283b4b3c791496bdf1c324144e25c00484d5c75db441f7d687b8
  Stored in directory: /root/.cache/pip/wheels/dd/54/e9/32364da8eee84a2b0b412394983c15add18816c507e90f02d8
Successfully built ctransformers


In [ ]:
# choose your champion
#model_id = "TheBloke/Llama-2-7B-GGML"
#model_id = "TheBloke/Llama-2-7B-chat-GGML"
#model_id = "TheBloke/Llama-2-13B-GGML"
model_id = "TheBloke/Llama-2-13B-chat-GGML"

In [ ]:
from ctransformers import AutoModelForCausalLM

config = {'max_new_tokens': 256, 'repetition_penalty': 1.1, 'temperature': 0.1, 'stream': True}

llm = AutoModelForCausalLM.from_pretrained(model_id,
                                           model_type="llama",
                                           #lib='avx2', for cpu use
                                           gpu_layers=130, #110 for 7b, 130 for 13b
                                           **config
                                           )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

llama-2-13b-chat.ggmlv3.q2_K.bin:   0%|          | 0.00/5.51G [00:00<?, ?B/s]

## LLAMA-2-7B-Chat tests

### Tokenizer

In [ ]:
prompt="""What are some coping mechanisms for dealing with depression?"""

In [ ]:
tokens = llm.tokenize(prompt)

In [ ]:
tokens

### Pipeline

In [ ]:
# 'pipeline' execution
llm(prompt, stream=False)

"\n\nCoping with depression can be a challenging and ongoing process, but there are several strategies that can help. Here are some effective coping mechanisms for dealing with depression:\n\n1. Seek professional help: One of the most important things you can do is seek professional help from a mental health professional such as a psychologist or psychiatrist. They can help you develop a treatment plan that includes therapy and/or medication.\n\n2. Stay active: Regular exercise can help improve mood and reduce symptoms of depression. Find activities that you enjoy, such as walking, jogging, swimming, or dancing.\n\n3. Connect with others: Social support is essential for managing depression. Reach out to friends and family members, or join a support group to connect with others who understand what you're going through.\n\n4. Practice mindfulness: Mindfulness techniques can help you focus on the present moment and reduce negative thoughts. Try meditation, yoga, or deep breathing exercise

### Generate with stream execution

In [ ]:
import time

start = time.time()
NUM_TOKENS = 0
MAX_TOKENS = 512  # Set the maximum number of tokens

print('-'*4 + 'Start Generation' + '-'*4)

for token in llm.generate(tokens):
    print(llm.detokenize(token), end='', flush=True)
    NUM_TOKENS += 1
    if NUM_TOKENS >= MAX_TOKENS:
        break

time_generate = time.time() - start

print('\n')
print('-'*4 + 'End Generation' + '-'*4)
print(f'Num of generated tokens: {NUM_TOKENS}')
print(f'Time for complete generation: {time_generate}s')
print(f'Tokens per second: {NUM_TOKENS / time_generate}')
print(f'Time per token: {(time_generate / NUM_TOKENS) * 1000}ms')


----Start Generation----


Coping with depression can be a challenging and ongoing process, but there are several strategies that can help. Here are some effective coping mechanisms for dealing with depression:

1. Seek professional help: One of the most important things you can do is seek professional help from a mental health professional such as a psychologist or psychiatrist. They can help you develop a treatment plan that includes therapy and/or medication.

2. Stay active: Regular exercise can help improve mood and reduce symptoms of depression. Find activities that you enjoy, such as walking, jogging, swimming, or dancing.

3. Connect with others: Social support is essential for managing depression. Reach out to friends and family members, or join a support group to connect with others who understand what you're going through.

4. Practice mindfulness: Mindfulness techniques such as meditation and deep breathing can help you focus on the present moment and reduce negative though

 with

 different

 strateg

ies

...

 with

 the

 more

 so

 be

----End Generation----
Num of generated tokens: 512
Time for complete generation: 28.844975471496582s
Tokens per second: 17.750058428925943
Time per token: 56.33784271776676ms
